<a href="https://colab.research.google.com/github/JoseReveco/MinorMachineLearning/blob/main/Proyecto%20Final%20ML/%20Entrega%2002%20-%20Scrub/%20Entrega_02_Scrub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Proyecto Final - Reconocimiento de Actividad Humana (HAR)


In [271]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder

###Entrega 01 - OBTAIN

Incluir todo el código de la Etapa 01 - OBTAIN creada en la semana anterior

In [272]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/WISDM_ar_v1.1_raw.txt'
# Agregamos una columna demas debido a que algunas filas del dataset poseen un campo vacio extra ( un septimo), luego la eliminamos.
data = pd.read_csv(path, header=None, names=["user", "activity" ,"timestamp", "x-acceleration", "y-accel", "z-accel", "del"])
data = data.drop('del', axis=1)
# Panda toma el ; al final de cada fila, lo eliminamos con:
data["z-accel"] = data["z-accel"].str[:-1]
df = pd.DataFrame(data = data)
df.count(axis=0)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


user              1098204
activity          1098204
timestamp         1098204
x-acceleration    1098204
y-accel           1098204
z-accel           1098203
dtype: int64

###Entrega 02 - SCRUB

Eliminar filas con datos faltantes

*Uno de los pasos iniciales en la limpieza de datos consiste en eliminar aquellas filas que contengan datos faltantes. Estas líneas generalmente no aportan información y pueden complicar la etapa de entrenamiento y validación de muchos modelos predictivos.*


In [273]:
# Quitamos los datos faltantes con dropna, luego contamos las filas con count(axis=0)
df = df.dropna()
df.count(axis=0)


user              1098203
activity          1098203
timestamp         1098203
x-acceleration    1098203
y-accel           1098203
z-accel           1098203
dtype: int64

Agregar al DataFrame los nombres de las columnas 

*En ocasiones al crear los data frames el sistema asigna nombres genéricos y correlativos a las columnas. Es recomendable renombrar las columnas por nombres significativos para favorecer el análisis posterior de los datos.*

In [274]:
# Le cambiamos los nombres a las columnas para que queden con el formato pedido
df = df.rename(columns={'user':'user','activity':'activity', 'timestamp':'time', 'x-acceleration':'x', 'y-accel':'y', 'z-accel':'z'})

Visualizar las 5 primeras filas del DataFrame (.head())

*Siempre es buena medida visualizar algunas líneas del dataframe para tener una idea concreta del contenido.*


In [275]:
# Mostramos las 5 primeras filas
df.head()

,user,activity,time,x,y,z
0,33,Jogging,49105962326000,-0.694638,12.680544,0.50395286
1,33,Jogging,49106062271000,5.012288,11.264028,0.95342433
2,33,Jogging,49106112167000,4.903325,10.882658,-0.08172209
3,33,Jogging,49106222305000,-0.612916,18.496431,3.0237172
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


Buscar datos nulos y eliminarlos si es que existen

In [276]:
#buscar nulos, nos da todos falsos
df.isnull().any()


user        False
activity    False
time        False
x           False
y           False
z           False
dtype: bool

Visualizar los tipo de datos a los que pertenece cada columna (.info())

*Es necesario tener un detalle de cuantas variables están considerando y el tipo de datos que pertenecen para verificar si es posible utilizarla directamente para el modelo predictivo o si es necesario realizar una transformación del tipo de dato.*


In [277]:
# Mostramos la informacion del df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   user      1098203 non-null  int64  
 1   activity  1098203 non-null  object 
 2   time      1098203 non-null  int64  
 3   x         1098203 non-null  float64
 4   y         1098203 non-null  float64
 5   z         1098203 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 58.7+ MB


Cambiar el tipo de datos de las variables predictoras a valores reales, para realizar la clasificación 

*Muchas de las variables contenidas en el DataFrame originalmente se encuentran en el tipo de dato 'object'. En ese formato las variables no pueden ser interpretadas como vectores. Recordemos que la mayoría de los modelos de Machine Learning estudiados operan en el espacio vectorial. Por esta razón las variables que formarán parte del modelo predictivo deben ser transformadas a un formato numérico como 'Float' o 'integer'.*

Para este proyecto en particular se pide transformar  al tipo de dato Float, las variables del acelerómetro en el eje $x$, $y$ y $z$



In [278]:
# Al intentar hacer la transformacion nos da el siguiente error:
df[['x', 'y', 'z']] = df[['x', 'y', 'z']].apply(pd.to_numeric)



ValueError: ignored

In [246]:
# Los datos vienen con errores, los corregiriemos:
# Para esto pasamos la columna z a String
df['z'] = df['z'].astype(str)
# Ahora que es un String buscamos todas las filas que poseen ";" en la columna z. 
mask = df.z.apply(lambda x: ';' in x)
df1 = df[mask]
df1.head()
# Estas filas son las que nos darán error:

,user,activity,time,x,y,z
134633,13,Downstairs,3498732310000,5.13,10.42,4.671779;1
197686,27,Jogging,10814072309000,4.21,-0.65,-0.6537767;2
273260,18,Jogging,0,0.00,0.00,0.0;1
371928,16,Downstairs,7160742320000,4.79,7.31,-4.0588636;
711479,30,Standing,1729732341000,-1.80,9.28,3.79;3


In [279]:
# Las modificamos con:
df.loc[134633,'z'] = 4.671779
df.loc[197686,'z'] = -0.6537767
df.loc[273260,'z'] = 0.0
df.loc[371928,'z'] = -4.0588636
df.loc[711479,'z'] = 3.79

# Y hacemos la conversión nuevamente con:
df[['x', 'y', 'z']] = df[['x', 'y', 'z']].apply(pd.to_numeric)

Imprimir los tipos de datos de las variables del DataFrame

*Se recomienda volver a ejecutar el comando info para verificar que efectivamente las variables cambiaron su tipo de dato y que ahora se pueden procesar con los modelos predictivos.*


In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1098203 entries, 0 to 1098203
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   user      1098203 non-null  int64  
 1   activity  1098203 non-null  object 
 2   time      1098203 non-null  int64  
 3   x         1098203 non-null  float64
 4   y         1098203 non-null  float64
 5   z         1098152 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 58.7+ MB


Eliminar las columnas que no se utilizaran para la clasificación

*En este caso deberíamos conservar la actividad respectiva y las variables asociadas al acelerómetro*


In [281]:
# Eliminamos las columnas user y time con drop
df = df.drop(['user', 'time'], axis=1)



Utilizar el comando *value_counts* para obtener un detalle de cuantas instancias se tienen por actividad. El objetivo consiste en identificar la presencia del balance o desbalance en las categorías a modelar


In [282]:
# Usamos value_counts para ver cuantos tipos existen en la columna activity
df['activity'].value_counts()

Walking       424397
Jogging       342176
Upstairs      122869
Downstairs    100427
Sitting        59939
Standing       48395
Name: activity, dtype: int64

Este proyecto considera datos con un desbalance de clases. Es necesario balancearlos para que los modelos operen de manera adecuada. Existen muchas maneras de realizar un balance, pero en este proyecto utilizaremos una estrategia sencilla e intuitiva. Simplemente debe identificar la actividad que contenga el número menor de instancias. Se debe realizar un muestreo para las otras actividades utilizando ese valor. De esta manera todas las clases tendrán la misma cantidad de instancias. 



In [283]:
# Balanceamos la columna recorriendo cada tipo de actividad y haciendo un sample con la cantidad menor
df = df.groupby('activity', as_index=False).apply(lambda x: x.sample(n=48395)).reset_index(drop=True)

Establecer el número de instancias y dimensiones del data frame resultante *(shape)*


In [284]:
# Realizamos el shape para comprobar que vamos bien.
df.shape

(290370, 4)

Volver a imprimir la cantidad de instancias por categoría para verificar que todo esté en orden. *(value_counts)*

In [285]:
# Comprobamos el balanceo
df['activity'].value_counts()

Downstairs    48395
Jogging       48395
Sitting       48395
Standing      48395
Upstairs      48395
Walking       48395
Name: activity, dtype: int64

Volver a visualizar las cinco primeras líneas del DataFrame para revisar que todo esté en orden *(head)*


In [286]:
# Nuevamente usamos head
df.head()

,activity,x,y,z
0,Downstairs,8.62,5.09,0.300000
1,Downstairs,1.31,10.99,-2.833032
2,Downstairs,2.68,3.87,1.270000
3,Downstairs,5.18,8.16,1.307553
4,Downstairs,3.57,10.99,2.914754


Separar las variables predictoras (X) de las categorias o etiquetas (y)

In [287]:
# Las predictoras son las x, y, z y las etiquetas son las actividades

X = df[["x", "y", "z"]]
y = df['activity']
X.head()


,x,y,z
0,8.62,5.09,0.300000
1,1.31,10.99,-2.833032
2,2.68,3.87,1.270000
3,5.18,8.16,1.307553
4,3.57,10.99,2.914754


Estandarizar los datos numericos (*StandardScaler)*

*Para el caso de la estandarización de los datos se pueden usar distintas técnicas, como la normalización 01, estandarización entre -1 y 1 o el denominado Z-Score. En este caso usaremos la función Standard Scaler de Python con sus valores por defecto.*


In [288]:
# Utilizamos StandarScaler para transformar los valores de X
scaled_features = StandardScaler().fit_transform(X.values)
# Los transformamos a un DataFrame, le damos los mismos nombres a las columnas usando columns e index, igualandolo a X.
scaled_features_df = pd.DataFrame(scaled_features, index=X.index, columns=X.columns)
scaled_features_df.head()

,x,y,z
0,1.375762,-0.314025,-0.224152
1,0.142539,0.683717,-0.909590
2,0.373663,-0.520338,-0.011937
3,0.795422,0.205139,-0.003722
4,0.523809,0.683717,0.347898


Como último paso debe reconstruir el dataframe pegando las columnas de los variables predicadores (X) y las categorías (y)


In [291]:
#Unimos ambos df con concat, utilizando un inner join.
df_final = pd.concat([y, scaled_features_df], axis=1, join='inner')
df_final.head()

,activity,x,y,z
0,Downstairs,1.375762,-0.314025,-0.224152
1,Downstairs,0.142539,0.683717,-0.909590
2,Downstairs,0.373663,-0.520338,-0.011937
3,Downstairs,0.795422,0.205139,-0.003722
4,Downstairs,0.523809,0.683717,0.347898


Referencias:
1. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
2. https://stackoverflow.com/questions/15891038/change-column-type-in-pandas
3. https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
4. https://www.analyticslane.com/2019/03/25/como-eliminar-columnas-y-filas-en-un-dataframe-pandas/
5. https://stackoverflow.com/questions/22391433/count-the-frequency-that-a-value-occurs-in-a-dataframe-column
6. https://stackoverflow.com/questions/52935324/make-dataframe-balanced-with-respect-to-a-specific-column
7. https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
8. https://www.geeksforgeeks.org/how-to-combine-two-dataframe-in-python-pandas/